<a href="https://colab.research.google.com/github/Hadarlack/Hadarlack/blob/master/foodType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the required libraries

In [0]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

# Model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder



# Dataset Loading and Examining


In [0]:
del data



In [0]:
del df
del df1
del df2

In [0]:
print ("Read Dataset ... ")

file=files.upload()
data=pd.read_csv("Nutrino Food Label Challenge.csv")



In [0]:
data.head()

In [0]:
df=data # make a copy for convenience 
print("All Data Shape : ", df.shape)


In [0]:
sns.countplot(y=df['foodType'], order=df['foodType'].value_counts().reset_index()["index"]) # y distribution
plt.title("foodType Distribution")
plt.gcf().set_size_inches(15,10)
plt.show()

 **imbalanced** **calsses!** **fix it later!**


In [0]:
# Categories with one or two samples only - maybe combine them later
# df2=df2[df2.foodType != 'Pizza']
# df2=df2[df2.foodType != 'Crepes']
# df2=df2[df2.foodType != 'Pie']
# df2=df2[df2.foodType != 'Legumes']
# df2=df2[df2.foodType != 'EnergyBars']
# df2=df2[df2.foodType != 'MockMeats']



Seperate ingridients : Since Each ingredient is an import feature for the food type, I expand each ingredient into its own variable so that the model can understand whether each ingredient is present or not.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
list(df['ingredients'].values.tolist()) 


In [0]:
df['ingredients'].head()

In [0]:
 vect = CountVectorizer(tokenizer=lambda x: [i.strip() for i in x.split('//')], lowercase=False) # Convert a collection of text documents to a matrix of token counts


In [0]:
dummies = vect.fit_transform(df['ingredients']) 


In [0]:
 print((pd.DataFrame(dummies.toarray(),columns = vect.get_feature_names())))

In [0]:
vect.get_feature_names()

In [0]:
df1 = pd.DataFrame(dummies.todense(),columns=vect.get_feature_names())


In [0]:
list(df1.values.tolist()) 


In [0]:
df1.head()

In [0]:
df2=pd.concat([df,df1],axis=1)


In [0]:
df2.head()

In [0]:
print("All Data Shape: ", df2.shape)


In [0]:
print("Vocab Length: ", len(vect.get_feature_names())) # 6 features for 3 boolean+name+foodType+ingredients


In [0]:
print("Number of Predictors: ", df.shape[0]) # raws train+test


---------------

# Predicting foodType

--------------------------------------

Prepare for Modeling 

In [0]:
df2['GlutenFree'] = pd.get_dummies(df2['GlutenFree'],drop_first=False)
df2['Vegetarian'] = pd.get_dummies(df2['Vegetarian'],drop_first=False)
df2['Vegan'] = pd.get_dummies(df2['Vegan'],drop_first=False)

In [0]:
df2['foodType'] = pd.Categorical(data['foodType'])

In [0]:
df2=df2.drop('ingredients',axis=1) # already processed
df2=df2.drop('name',axis=1) # not helpfull



In [0]:
X = df2.loc[:, df2.columns != 'foodType']
y = df2.loc[:, df2.columns == 'foodType']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape,y_train.shape)

**fix imbalanced classes**

In [0]:
from imblearn.over_sampling import RandomOverSampler 

ros = RandomOverSampler(random_state=42)

print(X_train.shape,y_train.shape)

X_res, y_res = ros.fit_resample(X_train, y_train) # upsampling only the training data
print(X_res.shape,y_res.shape) # train was 800 and now 7008

In [0]:
# from imblearn.over_sampling import SMOTE

# os = SMOTE(random_state=0,ratio='minority',k_neighbors=2) #upsampling

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# # columns = X_train.columns
# os_data_X,os_data_y=os.fit_sample(X_train, y_train) # upsampling only the training data
# # X_train = pd.DataFrame(data=os_data_X,columns=columns )
# # y_train= pd.DataFrame(data=os_data_y,columns=['y'])
# print(os_data_X.shape,os_data_y.shape)

In [0]:
# print("length of oversampled data is ",len(X_train))
# print("Number of no Vegans in oversampled data",len(y_train[y_train['y']==0]))
# print("Number of Vegans in oversampled data",len(y_train[y_train['y']==1]))
# print("Proportion of no Vegans data in oversampled data is ",len(y_train[y_train['y']==0])/len(X_train))
# print("Proportion of Vegans data in oversampled data is ",len(y_train[y_train['y']==1])/len(X_train))


In [0]:
X_train.head()

# Logistic Regression

This is a multiclass problem, so here I use the ovr one-over-rest method.

In [0]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(multi_class='ovr')
logmodel.fit(X_res,y_res)
predictions = logmodel.predict(X_test)

In [0]:
#   # Train multi-classification model with logistic regression - another option
#   from sklearn import linear_model
  
#   lr = linear_model.LogisticRegression()
#   lr.fit(X_res, y_res)
 
    

In [0]:
# # Train multinomial logistic regression model - another option
# mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_res, y_res)
# # mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train, y_train)

   

In [0]:
    from sklearn import metrics
    
    print ("Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_res, logmodel.predict(X_res)))
    print ("logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, logmodel.predict(X_test)))
    
#     print ("Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_res, mul_lr.predict(X_res)))
#     print ("Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, mul_lr.predict(X_test)))

**Over Fitting**

# Evaluation

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [0]:
# print(y_test)

In [0]:
# Evaluate using Cross Validation - on train set

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

seed=7
num_instances = len(X_res)

kfold = model_selection.KFold(n_splits=10, random_state=seed)
# model = LogisticRegression()
results = model_selection.cross_val_score(logmodel, X_res, y_res, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [0]:
# # Evaluate using Leave One Out Cross Validation


# num_folds = 5
# num_instances = len(X_res)
# loocv = model_selection.LeaveOneOut()
# # model = LogisticRegression()
# results = model_selection.cross_val_score(logmodel, X_res, y_res, cv=loocv)
# print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [0]:
# LogisticRegression().get_params().keys()

In [0]:
score = cross_validate(logmodel, X_res, y_res, return_train_score=False)
score["train_score"].mean()

In [0]:
submission = logmodel.predict(X_test)
score = cross_validate(logmodel, X_test, y_test, return_train_score=False)
score["test_score"].mean()


In [0]:
print(logmodel.predict(X_test))

In [0]:
# data555 = {'y_Predicted': submission,
#         'y_Actual':    y_test
#         }

# conf = pd.DataFrame(data555, columns=['y_Actual','y_Predicted'])
# print (conf)

In [0]:
print("Number of foodTypes in train: ", y_train.nunique())

In [0]:
print("Number of foodTypes in test: ", y_test.nunique())

In [0]:
# print( conf.nunique())

In [0]:
# print(conf['y_Predicted'].unique()) # the model predictes only the 9 most common values 

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.linear_model import LogisticRegression
from pandas import Series, DataFrame

In [0]:
#classifier = LinearSVC(C=0.80, penalty="l2", dual=False)
parameters = {'C':[1, 10]}
#clf = LinearSVC()
clf = LogisticRegression()

classifier = GridSearchCV(clf, parameters)

classifier=classifier.fit(X_train,y_train)

predictions=classifier.predict(X_test)

In [0]:
# Evaluate using Cross Validation

from sklearn import model_selection

seed=7
num_instances = len(X_train)

kfold = model_selection.KFold(n_splits=10, random_state=seed)
# model = LogisticRegression()
results = model_selection.cross_val_score(clf, X_train, y_train, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [0]:
print(classification_report(y_test,predictions))